In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

plt.style.use('ggplot')

In [59]:
url = "https://raw.githubusercontent.com/HipsterVizNinja/random-data/main/Music/hot-100/Hot%20100.csv"
raw_df = pd.read_csv(url)

In [65]:
df_singles = raw_df.groupby('song_id').max()

In [98]:
df_singles.shape

(30444, 12)

In [102]:
df_artists = df_singles.drop_duplicates(subset=['performer'])

In [104]:
df_artists.shape

(10499, 12)

In [66]:
df_singles.sort_values('chart_date', inplace = True)

In [67]:
df_sample = df_singles.tail(50)

In [68]:
df_sample.shape

(50, 12)

In [95]:
df_sample.head(6)

,chart_position,chart_date,song,performer,instance,time_on_chart,consecutive_weeks,previous_week,peak_position,worst_position,chart_debut,chart_url
song_id,,,,,,,,,,,,
Deck The HallsNat King Cole,47,2023-01-07,Deck The Halls,Nat King Cole,3.0,8,4.0,47.0,47,47,2020-12-26,https://www.billboard.com/charts/hot-100/2023-...
Something In The OrangeZach Bryan,77,2023-01-07,Something In The Orange,Zach Bryan,1.0,36,35.0,77.0,55,77,2022-05-07,https://www.billboard.com/charts/hot-100/2023-...
Someday At ChristmasLizzo,96,2023-01-07,Someday At Christmas,Lizzo,1.0,5,4.0,96.0,96,96,2022-12-10,https://www.billboard.com/charts/hot-100/2023-...
You ProofMorgan Wallen,49,2023-01-07,You Proof,Morgan Wallen,1.0,33,32.0,49.0,6,49,2022-05-28,https://www.billboard.com/charts/hot-100/2023-...
SnoozeSZA,74,2023-01-07,Snooze,SZA,1.0,3,2.0,61.0,29,74,2022-12-24,https://www.billboard.com/charts/hot-100/2023-...
Bebe DameFuerza Regida X Grupo Frontera,91,2023-01-07,Bebe Dame,Fuerza Regida X Grupo Frontera,1.0,1,NaN,NaN,91,91,2023-01-07,https://www.billboard.com/charts/hot-100/2023-...


In [49]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="f4a75b5cb56c4a8081a21ae8ff8508d1",
                                                           client_secret="bbd13ee6b5594135a0e8b8ca3b0b6c01"))

results = sp.search(q='artist:Kendrick Lamar', limit=1)

In [50]:
results['tracks']['items'][0]['artists'][0]['id']

'2YZyLoL8N0Wb9xBt1NhZWg'

In [53]:
results = sp.search(q='Stereophonics', limit=1)

In [54]:
results['tracks']['items'][0]['artists'][0]['id']

'21UJ7PRWb3Etgsu99f8yo8'

In [75]:
def spotify_artist_id(artist):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="f4a75b5cb56c4a8081a21ae8ff8508d1",
                                                           client_secret="bbd13ee6b5594135a0e8b8ca3b0b6c01"))
    query = 'artist:'+ artist
    response = sp.search(q=query, limit=1)
    artist_id =  response['tracks']['items'][0]['artists'][0]['id']
    return artist_id

In [105]:
res = {}
exceptions = []

for i, row in tqdm(df_artists.iterrows(), total=1):
    try:
        artist = row['performer']
        id_result = spotify_artist_id(artist)
        res[artist] = id_result
    except:
        exceptions.append(artist)

10499it [30:21,  5.76it/s]                   


In [106]:
len(exceptions)

3384

In [107]:
len(res)

7115

In [113]:
artist_ids = pd.DataFrame.from_dict(res, orient='index').reset_index()

In [143]:
artist_ids.head()

,artist,spotify_id
0,The Four Preps,5MsnoFODDc5nxWrjm99Zew
1,Frankie Vaughan,1ihw8Evlq7gPuoHW520STD
2,Chuck Berry,293zczrfYafIItmnmM3coR
3,The Ames Brothers,4oXaAEofJFedGweFqy5qiv
4,David Seville,4NI5R1DbX27ODY5JTqnv2l


In [135]:
len(artist_ids)

7115

In [115]:
artist_ids.rename(columns={'index':'artist', 0:'spotify_id'}, inplace=True)

In [117]:
from pathlib import Path
filepath = Path("./data/artist_ids.csv")
artist_ids.to_csv(filepath)

In [3]:
artist_ids = pd.read_csv("./data/artist_ids.csv")

In [4]:
artist_ids.drop(columns=["Unnamed: 0"], inplace = True, axis=1)

In [5]:
artist_ids.head()

,artist,spotify_id
0,The Four Preps,5MsnoFODDc5nxWrjm99Zew
1,Frankie Vaughan,1ihw8Evlq7gPuoHW520STD
2,Chuck Berry,293zczrfYafIItmnmM3coR
3,The Ames Brothers,4oXaAEofJFedGweFqy5qiv
4,David Seville,4NI5R1DbX27ODY5JTqnv2l


In [22]:
def spotify_artist_genres(artist_spotify_id):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="353cbd9851824c55a3db2c2cc9a6ddf9",
                                                           client_secret="c9d4c1084de74b8aa6aa3a713b2219c8"))
    
    result = sp.artist(artist_spotify_id)
    genres = result['genres']
    return genres

In [23]:
spotify_artist_genres("1ihw8Evlq7gPuoHW520STD")

['deep adult standards', 'merseybeat', 'rock-and-roll']

In [24]:
res_genres = {}
exceptions_genres = []

for i, row in tqdm(artist_ids.iterrows(), total=len(artist_ids)):
    try:
        artist = row['artist']
        artist_spotify_id = row['spotify_id']
        genres = spotify_artist_genres(artist_spotify_id)
        res_genres[artist] = genres
    except:
        exceptions_genres.append(artist)

100%|██████████| 7115/7115 [16:20<00:00,  7.25it/s]


In [31]:
artist_and_genres = pd.DataFrame.from_dict(res_genres, orient='index').reset_index()

In [32]:
artist_and_genres.rename(columns={'index':'artist'}, inplace=True)

In [33]:
artist_and_genres.head()

,artist,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,The Four Preps,deep adult standards,doo-wop,vocal harmony group,None,None,None,None,None,None,None,None,None,None,None,None
1,Frankie Vaughan,deep adult standards,merseybeat,rock-and-roll,None,None,None,None,None,None,None,None,None,None,None,None
2,Chuck Berry,blues rock,classic rock,rock,rock-and-roll,rockabilly,None,None,None,None,None,None,None,None,None,None
3,The Ames Brothers,adult standards,deep adult standards,vocal harmony group,None,None,None,None,None,None,None,None,None,None,None,None
4,David Seville,novelty,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [34]:
artist_and_genres.merge(artist_ids, right_on = 'artist', left_on= 'artist')

,artist,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,spotify_id
0,The Four Preps,deep adult standards,doo-wop,vocal harmony group,None,None,None,None,None,None,None,None,None,None,None,None,5MsnoFODDc5nxWrjm99Zew
1,Frankie Vaughan,deep adult standards,merseybeat,rock-and-roll,None,None,None,None,None,None,None,None,None,None,None,None,1ihw8Evlq7gPuoHW520STD
2,Chuck Berry,blues rock,classic rock,rock,rock-and-roll,rockabilly,None,None,None,None,None,None,None,None,None,None,293zczrfYafIItmnmM3coR
3,The Ames Brothers,adult standards,deep adult standards,vocal harmony group,None,None,None,None,None,None,None,None,None,None,None,None,4oXaAEofJFedGweFqy5qiv
4,David Seville,novelty,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4NI5R1DbX27ODY5JTqnv2l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7110,d4vd,bedroom pop,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5y8tKLUfMvliMe8IKamR32
7111,Sam Smith & Kim Petras,pop,uk pop,None,None,None,None,None,None,None,None,None,None,None,None,None,2wY79sveU1sp5g7SokKOiI
7112,"Metro Boomin, The Weeknd & 21 Savage",rap,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0iEtIxbK0KxaSlF7G42ZOp
7113,Stephen Sanchez,gen z singer-songwriter,pop,None,None,None,None,None,None,None,None,None,None,None,None,None,5XKFrudbV4IiuE5WuTPRmT


In [36]:
from pathlib import Path
filepath2 = Path("./data/artist_and_genres.csv")
artist_and_genres.to_csv(filepath2)

In [37]:
artist_and_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7115 entries, 0 to 7114
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  7115 non-null   object
 1   0       5779 non-null   object
 2   1       4502 non-null   object
 3   2       3605 non-null   object
 4   3       2703 non-null   object
 5   4       1987 non-null   object
 6   5       1418 non-null   object
 7   6       930 non-null    object
 8   7       569 non-null    object
 9   8       337 non-null    object
 10  9       195 non-null    object
 11  10      119 non-null    object
 12  11      67 non-null     object
 13  12      42 non-null     object
 14  13      16 non-null     object
 15  14      6 non-null      object
dtypes: object(16)
memory usage: 889.5+ KB


In [43]:
print(artist_and_genres[0].nunique())
print(artist_and_genres[1].nunique())
print(artist_and_genres[2].nunique())

814
669
533
